In [ ]:
from pygromos.files.topology import top
from pygromos.files.topology import ptp
from pygromos.files.coord import cnf

import copy
import numpy as np

from rdkit import Chem
from rdkit.Chem import AllChem

In [ ]:
input_dir = "input"
in_top1_path = input_dir+"/F313.top"
in_top2_path = input_dir+"/G078.top"

in_coord2_path = input_dir+"/G078_unitedatom_optimised_geometry.g96"
in_coord1_path = input_dir+"/F313_unitedatom_optimised_geometry.g96"

out_ptp_path = "./out/fun.ptp"
out_top_path = "./out/fun.top"
out_cnf_path = "./out/coord.g96"


## COORD

In [ ]:
cmol1 = cnf.Cnf(in_coord1_path)
cmol2 = cnf.Cnf(in_coord2_path)

In [ ]:
cmol2.POSITION

In [ ]:
cmol2.visualize()

In [ ]:
 cmol1.visualize()

### Atom Matching

In [ ]:
from rdkit.Chem import rdFMCS

def find_atom_mapping(cnfA:cnf.Cnf, cnfB:cnf.Cnf)->(dict,str):
    
    mol1 = Chem.MolFromPDBBlock(cnfA.get_pdb(rdkit_ready=True), removeHs=False)
    mol2 = Chem.MolFromPDBBlock(cnfB.get_pdb(rdkit_ready=True), removeHs=False)


    #CompareElements
    mcs = rdFMCS.FindMCS([mol1, mol2],
                         atomCompare=rdFMCS.AtomCompare.CompareAnyHeavyAtom,
                         bondCompare=rdFMCS.BondCompare.CompareOrderExact,
                         ringMatchesRingOnly=True, completeRingsOnly=True)
    #                    ringCompare=rdFMCS.RingCompare.StrictRingFusion)
    smartsString = mcs.smartsString

    ##MCS Match
    patt = Chem.MolFromSmarts(smartsString)  # smartsString
    mol1Match = [i+1 for i in mol1.GetSubstructMatch(patt)]
    mol2Match = [i+1 for i in mol2.GetSubstructMatch(patt)]

    atom_mappingAB = dict(zip(mol1Match, mol2Match))
    return atom_mappingAB, smartsString

atom_mappingAB, smart = find_atom_mapping(cnfA=cmol1, cnfB=cmol2)
len(atom_mappingAB), atom_mappingAB, smart

## COORDINATE Generation
### Coord Alignment

In [ ]:
def align_cnfs_with_MCS(cnfA:cnf.Cnf, cnfB:cnf.Cnf, atom_mappingAB:int)->(cnf.Cnf, cnf.Cnf):
    
    cnfA= copy.deepcopy(cnfA)
    cnfB= copy.deepcopy(cnfB)
    
    mol1 = Chem.MolFromPDBBlock(cnfA.get_pdb(rdkit_ready=True), removeHs=False)
    mol2 = Chem.MolFromPDBBlock(cnfB.get_pdb(rdkit_ready=True), removeHs=False)
    
    atom_mappingAB = {key-1:val-1 for key, val in atom_mappingAB.items()}
    AllChem.AlignMol(mol1, mol2, atomMap=list(atom_mappingAB.items()))


    conf2 = mol2.GetConformer()
    for ind, pos in enumerate(conf2.GetPositions()):
        cnfB.POSITION[ind].xp= pos[0]/10
        cnfB.POSITION[ind].yp= pos[1]/10
        cnfB.POSITION[ind].zp= pos[2]/10

    conf1 = mol1.GetConformer()
    for ind, pos in enumerate(conf1.GetPositions()):
        cnfA.POSITION[ind].xp= pos[0]/10
        cnfA.POSITION[ind].yp= pos[1]/10
        cnfA.POSITION[ind].zp= pos[2]/10
        
    return cnfA, cnfB

cmol1, cmol2 = align_cnfs_with_MCS(cnfA=cmol1, cnfB=cmol2, atom_mappingAB=atom_mappingAB)



### State merging

In [ ]:
cmol2.POSITION

In [ ]:
eucldean_dist = lambda x,y: np.sqrt((x.xp-y.xp)**2+(x.yp-y.yp)**2+(x.zp-y.zp)**2)

def merge_states(cnfA:cnf.Cnf, cnfB:cnf.Cnf, atomMatchingAB:dict, dist_tresh:float=0.09,_doNotChangeAtomType:bool=False, _doUpdateAtomMapping:bool=True)-> cnf.Cnf:#nm
        
    natoms_in_a=len(cnfA.POSITION)
    match_molA, match_molB = np.array(list(atom_mappingAB.items())).T
    atomMatchingAB = copy.deepcopy(atomMatchingAB)
    cmol_comb = copy.deepcopy(cnfA)    
    
    #reduce stateB to required atoms
    for pos in cnfB.POSITION:
        pos2s = [pos2 for pos2 in cnfA.POSITION  if(eucldean_dist(pos, pos2) < dist_tresh)]
        #print(pos.atomID)

        if(len(pos2s) > 0 and _doUpdateAtomMapping):#check distance
            found = False
            for p in pos2s:
                if(not(p.atomID in atomMatchingAB.keys() or pos.atomID in atomMatchingAB.values())):
                    #print("map_Coordinates", pos.atomID, {pos.atomID: p.atomID})
                    atomMatchingAB.update({p.atomID:pos.atomID})
                    found = True
                    break
            if(not (found or pos.atomID in atomMatchingAB.values())):
                #print("adding_Coordinates", pos.atomID)
                pos = copy.deepcopy(pos)
                pos.resID=2
                cmol_comb.POSITION.append(pos)      
            elif(not found):
                #print("already found", pos.atomID, pos.atomID in  atomMatchingAB.values())
                pass
        else:
            #print("adding_Coordinates", pos.atomID)
            pos = copy.deepcopy(pos)
            pos.resID=2
            cmol_comb.POSITION.append(pos)

    #print("LENE: ", len(atomMatchingAB), len(cmol_comb.POSITION))
    #print(cmol_comb.POSITION)
    #print(atomMatchingAB)
    
    #clean cmol_comb
    present_atoms={}
    ati = []
    atomID = 1
    for ind, pos in enumerate(cmol_comb.POSITION):
        #atoms present in coords, mapped to old initial ones
        tID = pos.atomID
        first_state = tID if (pos.resID == 1) else -1
        #print("first", first_state, pos.resID)
        if(first_state != -1):
            if(first_state in atomMatchingAB):
                #print("that if: ", atomMatchingAB[first_state], atomMatchingAB[first_state]+natoms_in_a)
                second_state = atomMatchingAB[first_state]+natoms_in_a
                ati.append(atomMatchingAB[first_state])
            else:
                second_state = -1
        else:
            #print("this if: ", pos.atomID, pos.atomID+natoms_in_a)
            second_state = pos.atomID+natoms_in_a
            ati.append(pos.atomID)

        pos.resID=1
        pos.atomID = atomID
        pos.resName=cmol1.POSITION[0].resName[:2]+cmol2.POSITION[0].resName[2:4]
        if(not _doNotChangeAtomType): pos.atomType = "C"
        present_atoms.update({pos.atomID: (first_state, second_state)})
        atomID+=1        
        
    #print(len(ati), sorted(ati))        
    return cmol_comb, present_atoms

cmol_comb, present_atoms = merge_states(cnfA=cmol1, cnfB=cmol2, atomMatchingAB=atom_mappingAB, dist_tresh=0.09)
#cmol_comb.write("fung.g96")
#cmol_comb.visualize()
present_atoms

In [ ]:
cmol_comb.visualize()

## Topology

In [ ]:
present_atoms

In [ ]:
top1 = top.Top(in_top1_path)
top2 = top.Top(in_top2_path)


In [ ]:
com_top = top2
com_top += top1
red_com_top = copy.deepcopy(com_top)

In [ ]:
#BONDS
bonds =[]
for bond in com_top.BOND:
    if(bond.IB in present_atoms and bond.JB in present_atoms):
        bonds.append(bond)
for bond in com_top.BONDH:
    if(bond.IB in present_atoms and bond.JB in present_atoms):
        bonds.append(bond)
red_com_top.BOND.content = bonds
red_com_top.BOND.NBON = len(bonds)

del red_com_top.BONDH


view = cmol_comb.visualize()

colors = ['green', 'blue', 'yellow', 'red', 'purple', 'black']
for i, bond in enumerate(red_com_top.BOND):
    view.setStyle({"serial": [bond.IB,  bond.JB]}, {"stick":{'color':colors[i%len(colors)]}})
view

In [ ]:


#BondAngles
angles =[]
for angle in com_top.BONDANGLE:
    if(angle.IT in present_atoms and angle.JT in present_atoms and angle.KT in present_atoms):
        angles.append(angle)

for angleH in com_top.BONDANGLEH:
    if(angleH.IT in present_atoms and angleH.JT in present_atoms and angleH.KT in present_atoms):
        angles.append(angleH)
        
red_com_top.BONDANGLE.content = angles
red_com_top.BONDANGLE.NTHE = len(angles)

if(hasattr(red_com_top, "BONDANGLEH")): del red_com_top.BONDANGLEH


view = cmol_comb.visualize()

colors = ['green', 'blue', 'yellow', 'red', 'purple', 'black']
for i, angle in enumerate(red_com_top.BONDANGLE):
    view.setStyle({"serial": [angle.IT,  angle.JT, angle.KT]}, {"stick":{'color':colors[i%len(colors)]}})
view

In [ ]:
com_top.DIHEDRALH

In [ ]:
red_com_top.DIHEDRAL

In [ ]:
present_atoms

In [ ]:
#Dihedrals
dihedrals =[]
for dih in com_top.DIHEDRAL:
    if(dih.IP in present_atoms and dih.JP in present_atoms and dih.KP in present_atoms and dih.LP in present_atoms):
        dihedrals.append(dih)

dih_type = type(dih)
for dihh in com_top.DIHEDRALH:
    if(dihh.IPH in present_atoms and dihh.JPH in present_atoms and dihh.KPH in present_atoms and dihh.LPH in present_atoms):
        dihedrals.append(dih_type(IP=dihh.IPH, JP=dihh.JPH, KP=dihh.KPH, LP=dihh.LPH, ICP=dihh.ICPH))
        
red_com_top.DIHEDRAL.content = dihedrals
red_com_top.DIHEDRAL.NPHI = len(dihedrals)

if(hasattr(red_com_top, "DIHEDRALH")): del red_com_top.DIHEDRALH

    
view = cmol_comb.visualize()

colors = ['green', 'blue', 'yellow', 'red', 'purple', 'black']
for i, tors in enumerate(red_com_top.DIHEDRAL):
    view.setStyle({"serial": [tors.IP,  tors.JP, tors.KP, tors.LP]}, {"stick":{'color':colors[i%len(colors)]}})
view

In [ ]:



#Impropers
impdihedrals =[]
for imp in com_top.IMPDIHEDRAL:
    if(imp.IQ in present_atoms and imp.JQ in present_atoms and imp.KQ in present_atoms and imp.LQ in present_atoms):
        impdihedrals.append(imp)
        
imp_type = type(imp)
for impH in com_top.IMPDIHEDRALH:
    if(impH.IQH in present_atoms and impH.JQH in present_atoms and impH.KQH in present_atoms and impH.LQH in present_atoms):
        impdihedrals.append(imp_type(IQ=impH.IQH, JQ=impH.JQH, KQ=impH.KQH, LQ=impH.LQH, ICQ=impH.ICQH))
        
red_com_top.IMPDIHEDRAL.content = impdihedrals
red_com_top.IMPDIHEDRAL.NQHI = len(impdihedrals)

if(hasattr(red_com_top, "IMPDIHEDRALH")): del red_com_top.IMPDIHEDRALH

view = cmol_comb.visualize()

colors = ['green', 'blue', 'yellow', 'red', 'purple', 'black']
for i, tors in enumerate(red_com_top.IMPDIHEDRAL):
    view.setStyle({"serial": [tors.IQ,  tors.JQ, tors.KQ, tors.LQ]}, {"stick":{'color':colors[i%len(colors)]}})
view

In [ ]:
vars(tors)

In [ ]:

#Mass, VDW, Charge
sol =[]
for atm in com_top.SOLUTEATOM:
    if(atm.ATNM in present_atoms):
        atm.MRES=1
        sol.append(atm)

red_com_top.SOLUTEATOM.content = sol 
red_com_top.SOLUTEATOM.NRP = len(sol)

"""
#crossDIHEDRAL
crds =[]
for crd in com_top.CROSSDIHEDRAL:
    print(vars(crd))
    if(crd.ATNM in present_atoms):
        crds.append(atm)

red_com_top.CROSSDIHEDRAL.content = crds 

#crossDIHEDRALH
crdsH =[]
for crdH in com_top.CROSSDIHEDRALH:
    print(vars(crdH))
    if(crdH.ATNM in present_atoms):
        crdsH.append(atm)

red_com_top.CROSSDIHEDRALH.content = crdsH
"""


In [ ]:
#REST
##resname meshing
molA_Name = com_top.RESNAME.content[1][0]
molB_Name = com_top.RESNAME.content[2][0]

red_com_top.RESNAME.content = [['1'], [molA_Name[:2]+molB_Name[:2]]]


## SOLUTEMOLECULES:
from collections import defaultdict
resi_atom = defaultdict(int)

for atom in red_com_top.SOLUTEATOM:
    resi_atom[atom.MRES]+=1

groups = [[str(len(resi_atom))], [str(resi_atom[res])+" " for res in resi_atom]]
red_com_top.SOLUTEMOLECULES.content =groups
red_com_top.TEMPERATUREGROUPS.content =groups
red_com_top.PRESSUREGROUPS.content =groups

pass

### Build PTP

In [ ]:
from pygromos.files.blocks.pertubation_blocks import TITLE, PERTBONDSTRETCH, PERTBONDSTRETCHH, atom_lam_pertubation_state_bond
from pygromos.files.blocks.topology_blocks import bondstretchtype_type, torsdihedraltype_type, impdihedraltype_type

### DummyTypes

In [ ]:
#Dummy Types
dummyBond = bondstretchtype_type(CB=0, CHB=0, B0=1)
dummyAngle = bondstretchtype_type(CB=0, CHB=0, B0=1)
dummyTors = torsdihedraltype_type(CP=0, PD=0, NP=1)
dummyImp = impdihedraltype_type(CQ=0, Q0=0)

dummyIAC = 21
dummyMass = -1
dummyCharge = 0.0

#Add to top
red_com_top.BONDSTRETCHTYPE.append(dummyBond)
red_com_top.BONDANGLEBENDTYPE.append(dummyAngle)
red_com_top.TORSDIHEDRALTYPE.append(dummyTors)
red_com_top.IMPDIHEDRALTYPE.append(dummyImp)


dummyBondInd = len(red_com_top.BONDSTRETCHTYPE)
dummyAngleInd = len(red_com_top.BONDANGLEBENDTYPE)
dummyTorsInd = len(red_com_top.TORSDIHEDRALTYPE)
dummyIMPsInd = len(red_com_top.IMPDIHEDRALTYPE)

red_com_top.BONDSTRETCHTYPE.NBTY = dummyBondInd
red_com_top.BONDANGLEBENDTYPE.NBTY = dummyAngleInd
red_com_top.TORSDIHEDRALTYPE.NPTY = dummyTorsInd
red_com_top.IMPDIHEDRALTYPE.NQTY = dummyIMPsInd


In [ ]:
def reduce_pertubations(pertubations):
    #reduce
    atom_keys = [k for k in vars(pertubations[0]) if("atom" in k)]
    stateAtoms = list(sorted(pertubations, key=lambda x: [vars(x)[k] for k in atom_keys]))
    skip = []
    reduced_pertubations = []
    for i,bond1 in enumerate(stateAtoms):
        if(bond1 in skip):
            continue
        else:
            #is there already a bond?
            attr = vars(bond1)
            for bond2  in stateAtoms[i+1:]:
                if(set([attr[k] for k in atom_keys]) == set([vars(bond2)[k] for k in atom_keys])):
                    skip.append(bond2)
                    replace_keys = [k for k in attr['STATES'] if (attr['STATES'][k] == dummyBondInd)]
                    if(len(replace_keys)>0):
                        replace_key = replace_keys[0]
                        attr['STATES'].update({replace_key: bond2.STATES[replace_key]})
                        break
                    else:
                        continue

            #something equal already there?
            if(attr['STATES'][0] == attr['STATES'][1] or 
               set([attr[k] for k in atom_keys]) in [set([vars(x)[k] for k in atom_keys]) for x in reduced_pertubations]):
                continue
            else:
                attr.update({"NR":len(reduced_stateAtoms)})
                new_pertAtomState = type(bond1)(**attr)
                reduced_pertubations.append(new_pertAtomState)
    return reduced_pertubations


def merge_tops_to_pertubations(tops, pertType, topBlockName, dummystateInd,  ):
    pertubations = []
    tops = [top1, top2]
    toffset = 0
    for tind, ttop in enumerate(tops):
        rearrange_dict = {val[tind]: key for key,val in present_atoms.items()}
        for top_bond in getattribute(ttop, topBlockName):
            
            tstates = {tind:top_bond.ICB}
            tstates.update({i:dummystateInd for i in range(len(tops)) if not i in tstates})
            pertubations.append(pertType(NR=len(pertBonds), 
                                         atomI=rearrange_dict[top_bond.IB+toffset],
                                         atomJ=rearrange_dict[top_bond.JB+toffset], 
                                         STATES=tstates)
                                )
        toffset = len(ttop.SOLUTEATOM)    
    
    return pertubations

### Bond

In [ ]:
pertbond =PERTBONDSTRETCH()

pertBonds = []
tops = [top1, top2]
toffset = 0
for tind, ttop in enumerate(tops):
    rearrange_dict = {val[tind]: key for key,val in present_atoms.items()}
    for top_bond in ttop.BOND:
        tstates = {tind:top_bond.ICB}
        tstates.update({i:dummyBondInd for i in range(len(tops)) if not i in tstates})
        pertBonds.append(atom_lam_pertubation_state_bond(NR=len(pertBonds), 
                                                         atomI=rearrange_dict[top_bond.IB+toffset],
                                                         atomJ=rearrange_dict[top_bond.JB+toffset], 
                                                         STATES=tstates)
                            )
    toffset = len(ttop.SOLUTEATOM)           

toffset = 0
for tind, ttop in enumerate(tops):
    rearrange_dict = {val[tind]: key for key,val in present_atoms.items()}
    for top_bond in ttop.BONDH:
        tstates = {tind:top_bond.ICB}
        tstates.update({i:dummyBondInd for i in range(len(tops)) if not i in tstates})
        pertBonds.append(atom_lam_pertubation_state_bond(NR=len(pertBonds), 
                                                         atomI=rearrange_dict[top_bond.IB+toffset],
                                                         atomJ=rearrange_dict[top_bond.JB+toffset], 
                                                         STATES=tstates)
                            )
    toffset = len(ttop.SOLUTEATOM)           



#reduce
reduced_stateAtoms = reduce_pertubations(stateAtoms)
        
pertbond.STATEATOMS = reduced_stateAtoms
pertbond.NPB = len(reduced_stateAtoms)

pertbond

### Angle

In [ ]:
from pygromos.files.blocks.pertubation_blocks import PERTBONDANGLE, PERTBONDANGLEH, atom_lam_pertubation_state_angle

In [ ]:
pertangle =PERTBONDANGLE()

pertAngles = []
tops = [top1, top2]

toffset = 0
for tind, ttop in enumerate(tops):
    rearrange_dict = {val[tind]: key for key,val in present_atoms.items()}
    for top_angle in ttop.BONDANGLE:
        tstates = {tind:top_angle.ICT}
        tstates.update({i:dummyAngleInd for i in range(len(tops)) if not i in tstates})
        pertAngles.append(atom_lam_pertubation_state_angle(NR=len(pertAngles), 
                                                         atomI=rearrange_dict[top_angle.IT+toffset],
                                                         atomJ=rearrange_dict[top_angle.JT+toffset], 
                                                         atomK=rearrange_dict[top_angle.KT+toffset], 
                                                         STATES=tstates)
                            )
    toffset = len(ttop.SOLUTEATOM)           
    
toffset = 0
for tind, ttop in enumerate(tops):
    rearrange_dict = {val[tind]: key for key,val in present_atoms.items()}
    for top_angle in ttop.BONDANGLE:
        tstates = {tind:top_angle.ICT}
        tstates.update({i:dummyAngleInd for i in range(len(tops)) if not i in tstates})
        pertAngles.append(atom_lam_pertubation_state_angle(NR=len(pertAngles), 
                                                         atomI=rearrange_dict[top_angle.IT+toffset],
                                                         atomJ=rearrange_dict[top_angle.JT+toffset], 
                                                         atomK=rearrange_dict[top_angle.KT+toffset], 
                                                         STATES=tstates)
                            )
    toffset = len(ttop.SOLUTEATOM)    
    
    

#reduce
reduced_pertAngles = reduce_pertubations(pertAngles)
            
pertangle.STATEATOMS = reduced_pertAngles
pertangle.NPA = len(reduced_pertAngles)

pertangle

### Dihedrals

In [ ]:
from pygromos.files.blocks.pertubation_blocks import PERTPROPERDIH, PERTPROPERDIHH, atom_lam_pertubation_state_dihedral

In [ ]:
pertproperdih =PERTPROPERDIH()

pertDihedrals = []
tops = [top1, top2]

toffset = 0
for tind, ttop in enumerate(tops):
    rearrange_dict = {val[tind]: key for key,val in present_atoms.items()}
    for top_dihedral in ttop.DIHEDRAL:
        tstates = {tind:top_dihedral.ICP}
        tstates.update({i:dummyTorsInd for i in range(len(tops)) if not i in tstates})
        pertDihedrals.append(atom_lam_pertubation_state_dihedral(NR=len(pertDihedrals), 
                                                             atomI=rearrange_dict[top_dihedral.IP+toffset],
                                                             atomJ=rearrange_dict[top_dihedral.JP+toffset], 
                                                             atomK=rearrange_dict[top_dihedral.KP+toffset],
                                                             atomL=rearrange_dict[top_dihedral.LP+toffset], 
                                                             STATES=tstates)
                            )
    toffset = len(ttop.SOLUTEATOM)   
    
toffset = 0
for tind, ttop in enumerate(tops):
    rearrange_dict = {val[tind]: key for key,val in present_atoms.items()}
    for top_dihedral in ttop.DIHEDRALH:
        tstates = {tind:top_dihedral.ICPH}
        tstates.update({i:dummyTorsInd for i in range(len(tops)) if not i in tstates})
        pertDihedrals.append(atom_lam_pertubation_state_dihedral(NR=len(pertDihedrals), 
                                                             atomI=rearrange_dict[top_dihedral.IPH+toffset],
                                                             atomJ=rearrange_dict[top_dihedral.JPH+toffset], 
                                                             atomK=rearrange_dict[top_dihedral.KPH+toffset],
                                                             atomL=rearrange_dict[top_dihedral.LPH+toffset], 
                                                             STATES=tstates)
                            )
    toffset = len(ttop.SOLUTEATOM)  

#reduce
reduced_pertDihedrals = reduce_pertubations(pertDihedrals)

    
pertproperdih.STATEATOMS = reduced_pertDihedrals
pertproperdih.NPD = len(reduced_pertDihedrals)

pertproperdih

### Improper Dihedrals

In [ ]:
from pygromos.files.blocks.pertubation_blocks import PERTIMROPERDIH, PERTIMROPERDIHH, atom_lam_pertubation_state_dihedral

In [ ]:
pertproperimpdih =PERTIMROPERDIH()

pertImpDihedrals = []
tops = [top1, top2]

toffset = 0
for tind, ttop in enumerate(tops):
    rearrange_dict = {val[tind]: key for key,val in present_atoms.items()}
    for top_imp in ttop.IMPDIHEDRAL:
        tstates = {tind:top_imp.ICQ}
        tstates.update({i:dummyIMPsInd for i in range(len(tops)) if not i in tstates})
        pertImpDihedrals.append(atom_lam_pertubation_state_dihedral(NR=len(pertImpDihedrals), 
                                                             atomI=rearrange_dict[top_imp.IQ+toffset],
                                                             atomJ=rearrange_dict[top_imp.JQ+toffset], 
                                                             atomK=rearrange_dict[top_imp.KQ+toffset],
                                                             atomL=rearrange_dict[top_imp.LQ+toffset], 
                                                             STATES=tstates)
                            )
    toffset = len(ttop.SOLUTEATOM)           

toffset = 0
for tind, ttop in enumerate(tops):
    rearrange_dict = {val[tind]: key for key,val in present_atoms.items()}
    for top_impH in ttop.IMPDIHEDRALH:
        tstates = {tind:top_impH.ICQH}
        tstates.update({i:dummyIMPsInd for i in range(len(tops)) if not i in tstates})
        pertImpDihedrals.append(atom_lam_pertubation_state_dihedral(NR=len(pertImpDihedrals), 
                                                             atomI=rearrange_dict[top_impH.IQH+toffset],
                                                             atomJ=rearrange_dict[top_impH.JQH+toffset], 
                                                             atomK=rearrange_dict[top_impH.KQH+toffset],
                                                             atomL=rearrange_dict[top_impH.LQH+toffset], 
                                                             STATES=tstates)
                            )
    toffset = len(ttop.SOLUTEATOM)        

reduced_pertImpDihedrals = reduce_pertubations(pertImpDihedrals)

pertproperimpdih.STATEATOMS = reduced_pertImpDihedrals
pertproperimpdih.NPD = len(reduced_pertImpDihedrals)

pertproperimpdih


### NONBONDEDS

In [ ]:
from pygromos.files.blocks.pertubation_blocks import PERTATOMPARAM, atom_lam_pertubation_state, pertubation_lam_state_nonbonded

In [ ]:
nonbonded = PERTATOMPARAM()

nonbondeds = []
tops = [top1, top2]
toffset = 0

dummyState =lambda m: pertubation_lam_state_nonbonded(IAC=dummyIAC, MASS=m, CHARGE=dummyCharge)

for atom in red_com_top.SOLUTEATOM:
    atom_states = present_atoms[atom.ATNM]
    tstates = {}
    for state, atomNR in enumerate(atom_states):
        if(atomNR == -1):
            tstates.update({state :dummyState(atom.MASS)})
        else:
            toffset = sum([len(tops[state].SOLUTEATOM) for i in range(state)])
            atom_nb = tops[state].SOLUTEATOM[atomNR-toffset-1]
            tstates.update({state:pertubation_lam_state_nonbonded(IAC=atom_nb.IAC, MASS=atom_nb.MASS, CHARGE=atom_nb.CG)})
                  

    nonbondeds.append(atom_lam_pertubation_state(NR=atom.ATNM,
                                                 RES=1,
                                                 NAME=atom_nb.PANM,
                                                 STATES=tstates)
                        )    
nonbonded.STATEATOMS = nonbondeds
nonbonded.NJLA = len(nonbondeds)
nonbonded.STATEIDENTIFIERS = ["state"+str(i) for i in range(len(tops))]


nonbonded

### BUild File

In [ ]:
red_com_top.RESNAME

In [ ]:
ptp_file = ptp.Ptp()


In [ ]:
ptp_file.add_block(block=TITLE("\tpertubations for "+"\n ".join(map(lambda x: "\t".join(x), red_com_top.RESNAME.content))))
ptp_file.add_block(block=pertbond)
ptp_file.add_block(block=pertangle)
ptp_file.add_block(block=pertproperdih)
ptp_file.add_block(block=pertproperimpdih)

ptp_file.add_block(block=nonbonded)

In [ ]:
ptp_file

# Check Pertubation top

In [ ]:
#reduce
stateAtoms = list(sorted(ptp_file.PERTBONDSTRETCH.STATEATOMS, key=lambda x: (x.atomI, x.atomJ)))
skip = []
reduced_stateAtoms = []
for i,bond1 in enumerate(stateAtoms):
    if(bond1 in skip):
        print('skip', bond1)
        continue
    else:
        #is there already a bond?
        attr = vars(bond1)
        for bond2  in stateAtoms[i+1:]:
            if(all([vars(bond1)[k]==vars(bond2)[k] for k in vars(bond1) if("atom" in k)])):
                print(bond1, bond2)
                skip.append(bond2)
                
                replace_keys = [k for k in attr['STATES'] if (attr['STATES'][k] == dummyBondInd)]
                if(len(replace_keys)>0):
                    replace_key = replace_keys[0]
                    print(attr, replace_key)

                    attr['STATES'].update({replace_key: bond2.STATES[replace_key]})
                    print(attr)
                    break
                else:
                    continue
        print(attr)
        if(attr['STATES'][0] == attr['STATES'][1]):
            continue
        attr.update({"NR":len(reduced_stateAtoms)})
        new_pertAtomState = type(bond1)(**attr)
        reduced_stateAtoms.append(new_pertAtomState)

ptp_file.PERTBONDSTRETCH.STATEATOMS =  reduced_stateAtoms             
ptp_file.PERTBONDSTRETCH.NPB = len(reduced_stateAtoms)
ptp_file.PERTBONDSTRETCH

In [ ]:
#write out all files
ptp_file_path = ptp_file.write(out_ptp_path)
top_file_path = red_com_top.write(out_top_path)
cnf_file_path = cmol_comb.write(out_cnf_path)

## Test Emin:


In [ ]:
from pygromos.files.gromos_system import Gromos_System
from pygromos.data.simulation_parameters_templates import template_emin
from pygromos.simulations.modules.preset_simulation_modules import emin
from pygromos.hpc_queuing.submission_systems.local import LOCAL as subSystem


In [ ]:
root_dir = "/home/bschroed/Documents/projects/PyGromosTools/examples/dev/singleTop"
template_emin

In [ ]:
sys_name = 'test'
grom_system = Gromos_System(in_cnf_path=cnf_file_path, in_top_path=top_file_path, in_imd_path=template_emin,
                            in_ptp_path=ptp_file_path,
                            system_name=sys_name, 
                            work_folder=root_dir+"/out")

In [ ]:
from pygromos.files.blocks.imd_blocks import PERTURBATION
lam =1
pert_block  =  PERTURBATION(NTG=1, NRDGL=0, RLAM=lam, DLAMT=0,
                            ALPHC=0.5, ALPHLJ=0.5, NLAM=2, NSCALE=0)
grom_system.imd.add_block(block=pert_block)

In [ ]:
grom_system.imd.INITIALISE.NTISHK = 0

help(grom_system.imd.INITIALISE)

In [ ]:
import os

subSys = subSystem(verbose=True)
emin_gromos_system, jobID = emin(in_gromos_system=grom_system, project_dir=root_dir, 
                                 submission_system=subSys, in_imd_path=None)

In [ ]:
emin_cnf = cnf.Cnf(root_dir+"/emin/simulation/emin_1/emin_1.cnf")

In [ ]:
emin_cnf.visualize()

## DONE

#### Dual Topology Approach

In [ ]:
def generate_dual_topology_approach(cnfA, cnfB, topA, topB, eds:bool=False):
    ##Atom Mapping
    atom_mappingAB, smart = find_atom_mapping(cnfA=cnfA, cnfB=cnfB)
    
    ##Coordinates
    cnfA, cnfB = align_cnfs_with_MCS(cnfA=cnfA, cnfB=cnfB, atom_mappingAB=atom_mappingAB)
    cnf_comb = copy.deepcopy(cnfA)
    #cnf_comb += cnfB # needs to be implemented
    
    for pos in cnfB.POSITION:
        cnf_comb.POSITION.append(pos)
    
    cnf_comb.supress_atomPosition_singulrarities()


    ##Top
    nAtoms_top1 = len(top1.SOLUTEATOM)
    top_comb = copy.deepcopy(top1)
    top_comb += top2
    
    ### Pertubation
    ptp_comb = ptp.Pertubation_topology()
    
    if(eds):
        from pygromos.files.blocks.pertubation_blocks import MPERTATOM
        from pygromos.files.blocks.pertubation_blocks import  atom_eds_pertubation_state, pertubation_eds_state

        tops = [topA, topB]
        dummyState = pertubation_eds_state(IAC=22, CHARGE=0)

        numStates=len(tops)
        IND = 1
        atom_states = []
        for top_ind, top in enumerate(tops):
            for atom in top.SOLUTEATOM:        
                states = {}
                for ctop in range(ntops):
                    if(ctop==top_ind):
                        states.update({ctop+1:pertubation_eds_state(IAC=atom.IAC, CHARGE=atom.CG)})
                    else:
                        states.update({ctop+1:dummyState})

                atom_ptp = atom_eds_pertubation_state(NR=IND, NAME=atom.PANM, STATES=states)
                atom_states.append(atom_ptp)
                IND+=1

        ptp_comb.add_block(block=MPERTATOM(NJLA=len(atom_states), NPTB=numStates, STATEATOMS=atom_states))

    else:
        from pygromos.files.blocks.pertubation_blocks import PERTATOMPARAM
        from pygromos.files.blocks.pertubation_blocks import  atom_lam_pertubation_state, pertubation_lam_state_nonbonded

        tops = [topA, topB]
        build_dummyState = lambda m: pertubation_lam_state_nonbonded(IAC=22, CHARGE=0, MASS=m)

        numStates=len(tops)
        IND = 1
        atom_states = []
        for top_ind, top in enumerate(tops):
            for atom in top.SOLUTEATOM:        
                states = {}
                for ctop in range(ntops):
                    if(ctop==top_ind):
                        states.update({ctop+1:pertubation_lam_state_nonbonded(IAC=atom.IAC, CHARGE=atom.CG, MASS=atom.MASS)})
                    else:
                        states.update({ctop+1:build_dummyState(atom.MASS)})

                atom_ptp = atom_lam_pertubation_state(NR=IND, RES=atom.MRES, NAME=atom.PANM, STATES=states,)
                atom_states.append(atom_ptp)
                IND+=1

        ptp_comb.add_block(block=PERTATOMPARAM(NJLA=len(atom_states), STATEATOMS=atom_states))
    
    return cnf_comb, top_comb, ptp_comb

In [ ]:
cnf_file, top_file, ptp_file = generate_dual_topology_approach(cnfA=cmol1, cnfB=cmol2, topA=top1, topB=top2)

In [ ]:
cnf_file.visualize()

In [ ]:
top_file.SOLUTEATOM

In [ ]:
ptp_file

#### Hybrid Topology Approach

In [ ]:
def generate_hybrid_topology_approach(cnfA, cnfB, topA, topB):
    ##Atom Mapping
    atom_mappingAB, smart = find_atom_mapping(cnfA=cnfA, cnfB=cnfB)
    
    ##Coordinates
    cnfA, cnfB = align_cnfs_with_MCS(cnfA=cnfA, cnfB=cnfB, atom_mappingAB=atom_mappingAB)
    cnf_comb, present_atoms = merge_states(cnfA=cmol1, cnfB=cmol2, atomMatchingAB=atom_mappingAB, dist_tresh=0.0, _doNotChangeAtomType=True, _doUpdateAtomMapping=True) #no distance collapsing
    
    ##Top
    
    
    ### Pertubation
    
    
    return cnf_comb
    #return cnf_comb, top_comb, ptp_comb

In [ ]:
cnf_file= generate_hybrid_topology_approach(cnfA=cmol1, cnfB=cmol2, topA=top1, topB=top2)

In [ ]:
cnf_file.visualize()

In [ ]:
cnf_file

#### Single Topology Approach

In [ ]:
def generate_single_topology_approach(cnfA, cnfB, topA, topB):
    ##Atom Mapping
    atom_mappingAB, smart = find_atom_mapping(cnfA=cnfA, cnfB=cnfB)
    
    ##Coordinates
    cnfA, cnfB = align_cnfs_with_MCS(cnfA=cnfA, cnfB=cnfB, atom_mappingAB=atom_mappingAB)
    cnf_comb,present_atoms = merge_states(cnfA=cmol1, cnfB=cmol2, atomMatchingAB=atom_mappingAB, dist_tresh=0.09) #no distance collapsing
    

    return cnf_comb #, top_comb, ptp_comb

In [ ]:
cnf_file= generate_single_topology_approach(cnfA=cmol1, cnfB=cmol2, topA=top1, topB=top2)

In [ ]:
cnf_file.visualize()

In [ ]:
cnf_file